WEBSCRAPING MERCADOLIBRE

In [239]:
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
import requests

Create the headers

In [240]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

Get links

In [241]:
productlinks = []
pages = ('', 'Desde_51_', 'Desde_101_', 'Desde_151_')
for x in pages:
    r = requests.get(f'https://listado.mercadolibre.com.co/consolas-videojuegos/consolas/sony/pplaystation-5_{x}NoIndex_True')
    soup = BeautifulSoup(r.content)
    productlist = soup.find_all('div', class_='ui-search-result__content-wrapper')
    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks.append(link['href'])

Get information by links

In [ ]:
productlist = []
for link in productlinks:
    url = link
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content)
    generalinfo = soup.find('div', class_='ui-pdp-header__info')
    try:
        price = soup.find('span', class_='andes-money-amount__fraction').text.strip()
    except:
        price = 0
    # color = soup.find('span', class_='picker-label-COLOR_SECONDARY_COLOR').text.strip()
    try:
        name = soup.find('h1').text.strip()
    except:
        name = soup.find('div', class_='ui-pdp-header__title-container').text.strip()
    try:
        reviews = generalinfo.find('span', class_='ui-pdp-review__amount').text.strip()
    except:
        reviews = 0
    try:
        rating = generalinfo.find('span', class_='ui-pdp-review__rating').text.strip()
    except:
        rating = 0
    try:
        state = soup.find('span', class_='ui-pdp-subtitle').text.strip()
    except:
        state = 'No Defined'
    try:
        discount = soup.find('span', class_='andes-money-amount__discount').text.strip()
    except:
        discount = 0
    tables = soup.find_all('table', class_='andes-table')
    product = {
        'name' : name,
        'price' : price,
        'rating' : rating,
        'reviews' : reviews,
        'state' : state,
        'discount' : discount,
        #'color' : color
    }
    rowstable = soup.find_all('tr', class_='andes-table__row ui-vpp-striped-specs__row')
    for row in rowstable:
        key = row.find('th').text
        value = row.find('td').text
        product[key] = value
    print(product, link)
    productlist.append(product)
print("Successfull Scrapper")

Convert the data into a df

In [ ]:
df = pd.DataFrame(productlist)
df.head()